In [1]:
from keras import models, layers
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from datagenerator import DataGenerator
import datetime
import pickle
from pyntcloud import PyntCloud
from mpl_toolkits.mplot3d import Axes3D

/Users/tristanbehrens/Development/python-venvs/venv-3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Hook to the proper dataset.

In [2]:
if os.path.exists("datasetpath.txt"):
    dataset_path = open("datasetpath.txt", "r").read().replace("\n", "")
else:
    dataset_path = "../data"

# Create a data-generator.

In [3]:
data_generator = DataGenerator(dataset_path=dataset_path, input_type="voxelgrid", output_targets=["height", "weight"])

print("jpg_paths", len(data_generator.jpg_paths))
print("pcd_paths", len(data_generator.pcd_paths))
print("json_paths_personal", len(data_generator.json_paths_personal))
print("json_paths_measures", len(data_generator.json_paths_measures))
print("QR-Codes:\n" + "\n".join(data_generator.qrcodes))

print("Done.")

jpg_paths 4511
pcd_paths 1360
json_paths_personal 40
json_paths_measures 74
QR-Codes:
SAM-02-003-01
SAM-GOV-001
SAM-GOV-002
SAM-GOV-003
SAM-GOV-004
SAM-GOV-005
SAM-GOV-008
SAM-GOV-011
SAM-GOV-012
SAM-GOV-013
SAM-GOV-014
SAM-GOV-023
SAM-GOV-025
SAM-GOV-026
SAM-GOV-033
SAM-GOV-034
SAM-GOV-035
SAM-GOV-036
SAM-GOV-037
SAM-GOV-038
SAM-GOV-041
SAM-GOV-042
SAM-GOV-043
SAM-GOV-044
SAM-GOV-099
SAM-SNG-011
SAM-SNG-012
SAM-SNG-013
SAM-SNG-014
SAM-SNG-015
SAM-SNG-016
SAM-SNG-021
SAM-SNG-036
SAM-SNG-066
SAM-SNG-067
SAM-SNG-072
SAM-SNG-091
SAM-SNG-096
Done.


# Do the train-validate-split.

In [4]:
# Ensure that the split is always the same.
random.seed(108)

# Filter and shuffle the QR-Codes.
qrcodes_shuffle = list(data_generator.qrcodes)
qrcodes_shuffle = [qrcode for qrcode in qrcodes_shuffle if qrcode.startswith("SAM-GOV")]
random.shuffle(qrcodes_shuffle)

# Do the split.
split_index = int(0.8 * len(qrcodes_shuffle))
qrcodes_train = qrcodes_shuffle[:split_index]
qrcodes_validate = qrcodes_shuffle[split_index:]
del qrcodes_shuffle

# Print statistics.
print("QR-Codes train:")
print(" ".join(qrcodes_train))
print("")
print("QR-Codes validate:")
print(" ".join(qrcodes_validate))
print("")
print("Done.")

QR-Codes train:
SAM-GOV-023 SAM-GOV-041 SAM-GOV-033 SAM-GOV-037 SAM-GOV-012 SAM-GOV-099 SAM-GOV-001 SAM-GOV-035 SAM-GOV-002 SAM-GOV-038 SAM-GOV-008 SAM-GOV-025 SAM-GOV-034 SAM-GOV-036 SAM-GOV-004 SAM-GOV-014 SAM-GOV-043 SAM-GOV-011 SAM-GOV-042

QR-Codes validate:
SAM-GOV-013 SAM-GOV-026 SAM-GOV-003 SAM-GOV-044 SAM-GOV-005

Done.


# Helper functions for network generation.

In [8]:
def create_dense_model(input_shape, output_size):
    
    model = models.Sequential(name="baseline-dense")
    model.add(layers.Flatten(input_shape=input_shape))
    model.add(layers.Dense(128, activation="relu"))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(output_size))
    
    return model


def create_voxnet_model_small(input_shape, output_size):
    """ See: http://dimatura.net/publications/3dcnn_lz_maturana_scherer_icra15.pdf """
    
    #Trainable params: 301,378
    model = models.Sequential(name="C7-F32-P2-C5-F64-P2-D512")
    model.add(layers.Reshape(target_shape=input_shape + (1,), input_shape=input_shape))
    model.add(layers.Conv3D(32, (7, 7, 7), activation="relu"))
    model.add(layers.MaxPooling3D((4, 4, 4)))
    model.add(layers.Conv3D(64, (5, 5, 5), activation="relu"))
    model.add(layers.MaxPooling3D((2, 2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation="relu"))
    model.add(layers.Dense(output_size))
    
    return model


def create_voxnet_model_big(input_shape, output_size):
    """ See: http://dimatura.net/publications/3dcnn_lz_maturana_scherer_icra15.pdf """
    
    # Trainable params: 7,101,442
    model = models.Sequential(name="C7-F64-P4-D512")
    model.add(layers.Reshape(target_shape=input_shape + (1,), input_shape=input_shape))
    model.add(layers.Conv3D(64, (7, 7, 7), activation="relu"))
    model.add(layers.MaxPooling3D((4, 4, 4)))
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation="relu"))
    model.add(layers.Dense(output_size))
    
    return model


def create_voxnet_model_homepage(input_shape, output_size):
    """ See: http://dimatura.net/publications/3dcnn_lz_maturana_scherer_icra15.pdf """
    
    # Trainable params: 916,834
    model = models.Sequential(name="VoxNetHomepage")
    model.add(layers.Reshape(target_shape=input_shape + (1,), input_shape=input_shape))
    model.add(layers.Conv3D(32, (5, 5, 5), strides=(2, 2, 2), activation="relu"))
    model.add(layers.Conv3D(32, (3, 3, 3), strides=(1, 1, 1), activation="relu"))
    model.add(layers.MaxPooling3D((2, 2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation="relu"))
    model.add(layers.Dense(output_size))
    
    return model

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 32, 32, 32, 1)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 14, 14, 14, 32)    4032      
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 12, 12, 12, 32)    27680     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 6, 6, 6, 32)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6912)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               884864    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total para

# Train several nets.

In [ ]:
# Parameters.
batch_size = 32
step_per_epoch = 32
epochs = 10
validation_steps = 8


# Create the models.
models_to_train = []
models_to_train.append(create_dense_model(data_generator.get_input_shape(), data_generator.get_output_size()))
models_to_train.append(create_voxnet_model_small(data_generator.get_input_shape(), data_generator.get_output_size()))
models_to_train.append(create_voxnet_model_big(data_generator.get_input_shape(), data_generator.get_output_size()))
models_to_train.append(create_voxnet_model_homepage(data_generator.get_input_shape(), data_generator.get_output_size()))

# Train the models.
histories = []
for model in models_to_train:

    # Some output.
    print("Training:", model.name)
    model.summary()

    # Compile the model.
    model.compile(
            optimizer="rmsprop",
            loss="mse",
            metrics=["mae"]
        )
    
    # Train the model.
    history = model.fit_generator(
        data_generator.generate(size=batch_size, qrcodes_to_use=qrcodes_train),
        steps_per_epoch=step_per_epoch,
        epochs=epochs,
        validation_data=data_generator.generate(size=batch_size, qrcodes_to_use=qrcodes_validate),
        validation_steps=validation_steps)
    
    histories.append(history)
    print("Done.")
    print("")

Training: baseline-dense
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               4194432   
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130       
Total params: 4,202,818
Trainable params: 4,202,818
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
32/32 [==============================] - 255s 8s/step - loss: 2856.1291 - mean_absolute_error: 41.1831 - val_loss: 2409.1778 - val_mean_absolute_error: 37.7164
Epoch 2/10
32/32 [==============================] - 65s 2s/step - lo

# Save the models.

In [ ]:
datetime_string = datetime.datetime.now().strftime("%Y%m%d-%H%M")

# Save the models and histories.    
for model in models_to_train:
    model_name = datetime_string + "-" + model.name
    model_path = os.path.join(model_name)
    #model.save(model_path + ".h5")
    #pickle.dump(history, open(model_path + ".p", "wb"))
    print("Model saved to:", model_path)
    
print("All saved.")

# Render histories.

In [ ]:
for index, history in histories:
    model = models_to_train[i]
    for key, data in history.history.items():
        plt.plot(data, label=model.name + "-" + key)
plt.show()
plt.close()